<a href="https://colab.research.google.com/github/693080593/research/blob/master/Error_Comparer_Char_Oil_Yield_041821_%5B3%5D_RF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import io 

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from google.colab import files
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


%matplotlib inline



In [ ]:
uploaded = files.upload()

Saving D.3 Error Test 2.csv to D.3 Error Test 2.csv


In [6]:
features = pd.read_csv('/content/Char Yield Effect on Biocrude 041821 Stratified Test.csv')
features.head(5)



,Lipid,Protein,Cellulose,Hemicellulose,Carbohydrate,Lignin,Ash,Char Yield,Bio-crude Oil Yield,Group
0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,1.300000,98.5000,1.0
1,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.852878,97.8678,1.0
2,100.0,0.0,0.0,0.0,0.0,0.0,0.0,1.300000,95.9000,1.0
3,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,95.0000,1.0
4,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,90.6000,1.0


In [7]:
# Labels are the values we want to predict
Original_labels = np.array(features['Bio-crude Oil Yield'])

# Remove the labels from the features
# axis 1 refers to the columns
Original_features= features.drop('Bio-crude Oil Yield', axis = 1)

group2 = Original_features.loc[:,'Group']

features_old, X_valid, labels, y_valid = train_test_split(Original_features, Original_labels, test_size = 0.1, stratify=group2, random_state=62)

features=features_old.drop('Group', axis=1)

# Saving feature names for later use

feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

In [9]:
#Code that runs simulations and puts errors for cell ids into a dataframe
from collections import Counter
from itertools import chain
def error_finder(tests):

  simulations_all = [] #holds keys, predictions, and errors 
  simulations_keys = [] #holds only the keys

  #Gets ids of all codes 
  feature_strings = [[str(x) for x in inner_int] for inner_int in features]
  code_ids = [["".join(inner_comps[0:len(feature_list)])] for inner_comps in feature_strings]
  #Might want to add ids if that is helpful 
  for i in range(tests):
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, shuffle=True, test_size = 0.20)

    test_features_variable = test_features.tolist()
    test_labels_variable= test_labels.tolist()

    params = {'n_estimators':120,'max_depth':13,'min_samples_leaf':1,'min_samples_split':2,'max_features':2,'random_state':33}
    
    rf = RandomForestRegressor(**params)
    rf.fit(train_features, train_labels);
    
    predictions = rf.predict(test_features)
    errors = abs(predictions - test_labels)
    

    #Converting the test features to strings so they can be concatenated
    test_feature_strings = [[str(x) for x in inner_int] for inner_int in test_features_variable]
    #Convert features to one cell by concatenating for the length of the feature list 
    new_strings_conc =  [["".join(inner_comps[0:len(feature_list)])] for inner_comps in test_feature_strings]

    #Convert test labels as strings 
    test_label_strings = [str(oil) for oil in test_labels_variable] 

    #Putting Yields, predictions, and errors with feature ID. Yields get appended
    for i in range(len(new_strings_conc)):
      new_strings_conc[i].append(test_label_strings[i])

    
    new_strings_conc_2 = [[''.join(inner_comps[0:2])] for inner_comps in new_strings_conc]

    for i in range(len(new_strings_conc)):    
      new_strings_conc_2[i].append(predictions[i])
      new_strings_conc_2[i].append(errors[i])

    #Separate entry if error is more than x (could change this)
    
    large_error_list = [] #contains keys, predictions, and errors 
    large_error_keys = []
    for i in range(len(new_strings_conc_2)):
      if new_strings_conc_2[i][-1] >=10:
        large_error_list.append(new_strings_conc_2[i])

        for keys in range(len(large_error_list)):
          large_error_keys.append(large_error_list[keys][0])

    simulations_all.append(large_error_list)
    simulations_keys.append(large_error_keys)   

    #Counts number of times a certain cell appears
    no_of_lists_per_name = Counter(chain.from_iterable(map(set, simulations_keys)))

    for name, no_of_lists in no_of_lists_per_name.most_common():
        if no_of_lists == 1:
            break # since it is ordered by count, once we get this low we are done
        #print(f"'{name}' is in {no_of_lists} lists")

  #Create a dataframe with IDs and number of errors
  error_df = pd.DataFrame.from_dict(no_of_lists_per_name, orient='index').reset_index()
  error_df = error_df.rename(columns={'index':'ID', 0:'count'})
  print(error_df)
  
  error_df.to_excel(excel_writer ="error_compare.xlsx")            
  from google.colab import files
  files.download('error_compare.xlsx')

error_finder(100)

                                                    ID  count
0                     5.920.90.00.048.90.024.527.029.0     17
1                     8.050.00.00.031.00.011.013.041.0     18
2                6.065.00.00.011.00.06.53.8281350.0391     19
3                      0.818.117.01.330.00.30.026.97.6     15
4    0.7379399.8080582.854670.00.00.06.5921.4338.26...     25
..                                                 ...    ...
118                   5.928.00.00.021.40.044.722.013.7      1
119                     4.660.10.00.018.50.07.62.127.7      1
120                18.1262.780.00.08.920.03.4221.046.0      1
121                  15.725.26.521.40.04.124.117.136.0      1
122                    28.520.10.00.044.70.06.74.149.4      1

[123 rows x 2 columns]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>